In [1]:
import zmq

# Create a ZeroMQ context
context = zmq.Context()

# Create a socket to communicate with the server
socket = context.socket(zmq.REQ)
server_ip = "10.203.232.86"
server_port = "5555"  # Make sure this matches the server's port
server_address = f"tcp://{server_ip}:{server_port}"

# Connect to the server
socket.connect(server_address)

# Send a string message to the server
message = "Hello, Server!"
print(f"Sending message to server: {message}")
socket.send_string(message)

# Wait for a reply from the server
reply = socket.recv_string()
print(f"Received reply from server: {reply}")

# Clean up
socket.close()
context.term()

Sending message to server: Hello, Server!
Received reply from server: Response from server


In [5]:
import numpy as np
import zmq

# Create a ZeroMQ context
context = zmq.Context()

# Create a socket to subscribe to the broadcast
socket = context.socket(zmq.SUB)
broadcast_ip = "10.203.232.86"
broadcast_port = "5588"  # Make sure this matches the broadcaster's port
broadcast_address = f"tcp://{broadcast_ip}:{broadcast_port}"

# Connect to the broadcast
socket.connect(broadcast_address)

# Subscribe to all messages
socket.setsockopt_string(zmq.SUBSCRIBE, '')

print("Listening for broadcast messages...")

try:
    while True:
        # Receive the message
        message = socket.recv()
        # Deserialize the numpy array
        array = np.frombuffer(message)
        print(f"Received array: {array}")
except KeyboardInterrupt:
    print("Stopped listening.")

# Clean up
socket.close()
context.term()

Listening for broadcast messages...


ValueError: buffer size must be a multiple of element size

In [31]:
import open3d as o3d
import numpy as np
import time 
# from scipy.spatial.transform import Rotation as R

# create a pyramid mesh representing a camera
vertices = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 0], [0, 1, 0], [0.5, 0.5, 1]])
triangles = np.array([[0, 1, 2], [0, 2, 3], [0, 1, 4], [1, 2, 4], [2, 3, 4], [3, 0, 4]])
camera_1 = o3d.geometry.TriangleMesh()
camera_1.vertices = o3d.utility.Vector3dVector(vertices)
camera_1.triangles = o3d.utility.Vector3iVector(triangles)

# create a second camera mesh
camera_2 = o3d.geometry.TriangleMesh()
camera_2.vertices = o3d.utility.Vector3dVector(vertices + np.array([2, 0, 0]))
camera_2.triangles = o3d.utility.Vector3iVector(triangles)
camera_2.paint_uniform_color([0.1, 0.1, 0.7])

# # create a third camera mesh
camera_3 = o3d.geometry.TriangleMesh()
camera_3.vertices = o3d.utility.Vector3dVector(vertices + np.array([4, 0, 0]))
camera_3.triangles = o3d.utility.Vector3iVector(triangles)
R = camera_3.get_rotation_matrix_from_xyz((0, 0, np.pi / 2))
camera_3.rotate(R, center=(0.5, 0.5, 0.5))

# create a coordinate axis
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[0, 1, 0])



# visualize the mesh
o3dviz = o3d.visualization.Visualizer()
o3dviz.create_window()
render_option = o3dviz.get_render_option()
render_option.mesh_show_back_face = True
render_option.mesh_show_wireframe = True
o3dviz.add_geometry(camera_1)
o3dviz.add_geometry(camera_2)
o3dviz.add_geometry(camera_3)
o3dviz.add_geometry(coord)



start_time = time.time()
frame_count = 0

try:
    while True:
        
    # changing the position and orientation of the mesh
        camera_1.vertices = o3d.utility.Vector3dVector(vertices + np.random.randn(5, 3) * 0.01)
        camera_1.triangles = o3d.utility.Vector3iVector(triangles)

        camera_2.vertices = o3d.utility.Vector3dVector(vertices + np.random.randn(5, 3) * 0.01 + np.array([2, 0, 0]))
        camera_2.triangles = o3d.utility.Vector3iVector(triangles)

        camera_3.vertices = o3d.utility.Vector3dVector(vertices + np.random.randn(5, 3) * 0.01 + np.array([4, 0, 0]))
        R = camera_3.get_rotation_matrix_from_xyz(np.random.randn(3) * 0.1)
        camera_3.rotate(R, center=(0.5, 0.5, 0.5))
        camera_3.triangles = o3d.utility.Vector3iVector(triangles)

        o3dviz.update_geometry(camera_1)
        o3dviz.update_geometry(camera_2)
        o3dviz.update_geometry(camera_3)
        o3dviz.poll_events()
        o3dviz.update_renderer()

        frame_count += 1
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            fps = frame_count / elapsed_time
            print(f"FPS: {fps:.2f}")


except KeyboardInterrupt:
    print("Stopped visualization.")
    o3dviz.destroy_window()


FPS: 20.05
FPS: 33.78
FPS: 40.11
FPS: 43.37
FPS: 46.23
FPS: 48.11
FPS: 49.33
FPS: 50.00
FPS: 51.09
FPS: 51.97
FPS: 52.60
FPS: 53.17
FPS: 53.29
FPS: 53.77
FPS: 54.12
FPS: 54.45
FPS: 54.71
FPS: 54.98
FPS: 55.24
FPS: 55.54
FPS: 55.70
FPS: 55.91
FPS: 56.06
FPS: 56.17
FPS: 56.26
FPS: 56.58
FPS: 56.70
FPS: 56.67
FPS: 56.92
FPS: 56.86
FPS: 57.11
FPS: 57.20
FPS: 57.28
FPS: 57.32
FPS: 57.44
FPS: 57.49
FPS: 57.54
FPS: 57.63
FPS: 57.69
FPS: 57.74
FPS: 57.73
FPS: 57.82
FPS: 57.90
FPS: 57.93
FPS: 58.01
FPS: 58.06
FPS: 58.09
FPS: 58.13
FPS: 58.16
FPS: 58.19
FPS: 58.23
FPS: 58.26
FPS: 58.30
FPS: 58.31
FPS: 58.31
FPS: 58.38
FPS: 58.41
FPS: 58.44
FPS: 58.46
FPS: 58.48
FPS: 58.50
FPS: 58.51
FPS: 58.55
FPS: 58.61
FPS: 58.61
FPS: 58.63
FPS: 58.64
FPS: 58.63
FPS: 58.65
FPS: 58.70
FPS: 58.72
FPS: 58.73
FPS: 58.75
FPS: 58.77
FPS: 58.78
FPS: 58.80
FPS: 58.81
FPS: 58.84
FPS: 58.83
FPS: 58.86
FPS: 58.87
FPS: 58.86
FPS: 58.89
FPS: 58.91
FPS: 58.92
FPS: 58.94
FPS: 58.95
FPS: 58.96
FPS: 58.97
FPS: 58.98
FPS: 58.99